In [1]:
from konstante import BAZA_MODELOV, URL_MODELI_AVTOMOBILOV

# Shranjevanje v bazo

In [33]:
import sqlite3
p = sqlite3.connect(BAZA_MODELOV)

In [34]:
def shrani_znamko_v_bazo(ime_znamke: str, povezava: str):
    c = p.cursor()
    c.execute(f"""INSERT INTO znamke(ime, povezava) VALUES("{ime_znamke}", "{povezava}")""")
    p.commit()
    c.execute("select last_insert_rowid();")
    return c.fetchall()[0][0]

def shrani_model_v_bazo(ime: str, id_znamke, tip, zacetno_leto, motorji, povezava):
    c = p.cursor()
    ime = ime.replace('"', '""')
    c.execute("INSERT INTO "
            "modeli(ime, znamka_id, tip, zacetno_leto, motorji, povezava)"
            f" VALUES(\"{ime}\", {id_znamke}, \"{tip}\", {zacetno_leto}, \"{motorji}\", '{povezava}')")
    p.commit()
    c.execute("select last_insert_rowid();")
    return c.fetchall()[0][0]

def shrani_razlicico_v_bazo(
    ime, id_modela, zacetno_leto, koncno_leto, vrsta_motorja, opis, povezava
):
    if koncno_leto == "Present":
        koncno_leto = 2022
    ime = ime.replace('"', '""')
    c = p.cursor()
    c.execute("INSERT INTO "
            "razlicice(ime, model_id, zacetno_leto, koncno_leto, vrsta_motorja, opis, povezava)"
            f" VALUES(\"{ime}\", {id_modela}, {zacetno_leto}, {koncno_leto}, '{vrsta_motorja}', \"{opis}\", '{povezava}')")
    p.commit()
    c.execute("select last_insert_rowid();")
    return c.fetchall()[0][0]

In [35]:
def odstrani_znamko_iz_baze(id: int):
    c = p.cursor()
    c.execute(f"DELETE FROM razlicice WHERE razlicice.model_id IN (SELECT modeli.id FROM modeli WHERE modeli.znamka_id IN (SELECT znamke.id FROM znamke WHERE znamke.id={id}));")
    c.execute(f"DELETE FROM modeli WHERE modeli.znamka_id IN (SELECT znamke.id FROM znamke WHERE znamke.id={id});")
    c.execute(f"DELETE FROM znamke WHERE znamke.id={id};")
    p.commit()

def ponastavi_bazo():
    ja = input("ali res želiš?(DA)")
    if ja == "DA" or ja == "da":
        c = p.cursor()
        c.execute("DELETE FROM znamke;")
        c.execute("DELETE FROM modeli;")
        c.execute("DELETE FROM razlicice;")
        c.execute("delete from sqlite_sequence where name='razlicice';")
        c.execute("delete from sqlite_sequence where name='modeli';")
        c.execute("delete from sqlite_sequence where name='znamke';")
        p.commit()

In [44]:
#ponastavi_bazo()

In [6]:
#odstrani_znamko_iz_baze(58)

# Iskanje

In [2]:
import requests
import re

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

In [3]:
def podatki_o_modelu_req(html: str):
    pod = re.search(
        r'<div class="padsides_20i newstext mgbot_20 fsz14"[\s\S]*?'
		r'(?:<strong>Generations:</strong>\s(?P<generacij>\d{1,2})<br>\s*?)?'
		r'(?:<strong>First production year:</strong>\s(?P<zacetno_leto>\d{4})<br>\s*?)?'
		r'(?:<strong>Engines:</strong>\s(?P<motorji>[^\n]*?)<br>\s*?)?'
        r'(?:<strong>Body style:</strong>\s(?P<tip>.*?)<br>)?\s*?</div>',
        html,
        flags=re.MULTILINE
    )
    return pod.groupdict()

def poisci_razlicice_req(p: str):
    html = requests.get(p, headers=headers).text
    r = []
    razlicice_html = re.findall(
        r'<div class="container carmodel clearfix"[\s\S]*?'
        r'<div class="col3width fl">',
        html,
        flags=re.MULTILINE
    )
    for razlicica in razlicice_html:
        ime_razlicice = re.search(r'<span class="col-red">(.*?)</span>', razlicica).group(1)
        try:
            obdobje = re.search(
                r'<p class="years"><a[\s\S]*?>(.*?)</a>\s*?</p>',
                razlicica,
                re.MULTILINE
            ).group(1)
        except:
            obdobje = ""
        try:
            zleto, kleto = obdobje.split(" - ")
        except:
            zleto = obdobje
            kleto = obdobje
        if kleto == "Present": kleto = 2022
        motorji = re.findall(
            r'<div class="col12width tocol1 mot clearfix">\s*?<div class="col1width fl"><strong class="upcase">(.*?):</strong></div>\s*?'
            r'<div class="col2width fl bcol-white">([\s\S]*?)</div>\s*?</div>',
            razlicica,
            re.MULTILINE
        )
        for motor in motorji:
            vrsta_motorja = motor[0].replace(" engines", "")
            motor_html = motor[1]
            for podrazlicica in re.findall(
                r'<p class="engitm">\s*?<a href="(.*?)".*?<span class="col-green2">(.*?)</span>',
                motor_html, re.MULTILINE
            ):

                povezava = podrazlicica[0]
                opis = podrazlicica[1]
                r.append({
                    "ime": ime_razlicice,
                    "zacetno_leto": zleto,
                    "koncno_leto": kleto,
                    "motor": vrsta_motorja,
                    "opis": opis,
                    "povezava": povezava,
                })
    return podatki_o_modelu_req(html), r

def poisci_modele_req(p: str):
    html = requests.get(p, headers=headers).text
    return [m.groupdict() for m in re.finditer(
        r'<div class="carmod clearfix[\s\S]*?'
        r'<a href="(?P<povezava>.*?)".*?<h4>(?P<ime>.*?)</h4>',
        html,
        re.MULTILINE
    )]

def poisci_znamke_req(p: str):
    html = requests.get(p, headers=headers).text
    return[
        z.groupdict() for z in re.finditer(
            r'itemtype="https://schema.org/Brand"><a href="(?P<povezava>.*?)"[\s\S]+?<span.*?itemprop="name">(?P<ime>.*?)</span>',
            html,
            re.MULTILINE
        )
    ]

In [7]:
def zavihek(f):
    def f1(*args, **kwargs):
        star_zavihek = args[0].current_window_handle
        args[0].switch_to.new_window('tab')
        args[0].get(args[1])
        r = f(*args, **kwargs)
        args[0].close()
        args[0].switch_to.window(star_zavihek)
        return r
    return f1

@zavihek
def poisci_razlicice(b, p: str):
    r = []
    for razlicica in b.find_elements(By.CSS_SELECTOR, "div[class*='container carmodel clearfix']"):
        ime_razlicice = razlicica.find_element(By.CSS_SELECTOR, "span[class*='col-red']").text
        try:
            obdobje = razlicica.find_element(By.CSS_SELECTOR, "p[class*='years']").text
        except:
            obdobje = ""
        try:
            zleto, kleto = obdobje.split(" - ")
        except:
            zleto = obdobje
            kleto = obdobje
        
        if kleto == "Present": kleto = 2022

        for motor in razlicica.find_elements(By.CSS_SELECTOR, "div[class*='col12width tocol1 mot clearfix']"):
            vrsta_motorja = razlicica.find_element(By.CSS_SELECTOR, "strong[class*='upcase']").text[:-1]

            for podrazlicica in motor.find_elements(By.CSS_SELECTOR, "p[class*='engitm']"):
                povezava = podrazlicica.find_element(By.CSS_SELECTOR, "a[class*='engurl semibold']").get_attribute("href")
                opis = podrazlicica.find_element(By.CSS_SELECTOR, "span[class*='col-green2']").text
                r.append({
                    "ime": ime_razlicice,
                    "zacetno_leto": zleto,
                    "koncno_leto": kleto,
                    "motor": vrsta_motorja,
                    "opis": opis,
                    "povezava": povezava,
                })
    return r

@zavihek
def podatki_o_modelu(b, p: str):
    podatki_o_modelu = b.find_element(
        By.CSS_SELECTOR,
        "div[class*='padsides_20i newstext mgbot_20 fsz14']"
    ).text
    pod = re.search(
        r"(?:Generations:\s(?P<generacij>\d{1,2})$)?"
        r"(?:\n?First produc.*?ar:\s(?P<prvo_leto>\d{4})$)?"
        r"(?:\n?Engines:\s(?P<motorji>[^\n]*?)$)?"
        r"(?:\n?Body style:\s(?P<tip>.*?)$)?",
        podatki_o_modelu,
        flags=re.MULTILINE
    )
    return pod.groupdict()

@zavihek
def poisci_modele(b, p: str):
    modeli = []
    for m in b.find_elements(By.CSS_SELECTOR, "div[class*='carmod clearfix']"):
        ime = m.find_element(By.TAG_NAME, "h4").text
        povezava = m.find_element(By.TAG_NAME, "a").get_attribute("href")
        podatki = podatki_o_modelu(b, povezava)
        modeli.append({
            "ime": ime,
            "zacetno_leto": podatki["prvo_leto"],
            "tip": podatki["tip"],
            "motorji": podatki["motorji"],
            "povezava": povezava,
        })
    return modeli

@zavihek
def poisci_znamke(b, p: str):
    znamke = []
    for z in b.find_elements(
        By.CSS_SELECTOR,
        "div[itemtype='https://schema.org/Brand']"
    ):
        ime = z.find_element(By.TAG_NAME, "h5").text
        povezava = z.find_element(By.TAG_NAME, "a").get_attribute("href")
        znamke.append({"ime": ime, "povezava": povezava})
    return znamke

## Testi funkcij

In [5]:
poisci_razlicice_req(
    "https://www.autoevolution.com/land-rover/discovery-sport/"
)

In [10]:
# poisci_modele(
#     brskalnik,
#     "https://www.autoevolution.com/ac/"
# )

In [11]:
# poisci_znamke(
#     brskalnik,
#     "https://www.autoevolution.com/cars/"
# )

# Skupno

In [12]:
# b = webdriver.Firefox(service=Service(GeckoDriverManager().install()))
# b.implicitly_wait(5)
# b.get(URL_MODELI_AVTOMOBILOV)
# time.sleep(3)
# # OK piškotki
# b.find_element(by=By.XPATH, value='/html/body/div[1]/div/div/div/div[2]/div/button[2]').click()


[WDM] - Downloading: 19.0kB [00:00, 4.87MB/s]                   


In [49]:
for znamka in poisci_znamke_req(URL_MODELI_AVTOMOBILOV)[100:]:
    id_znamke = shrani_znamko_v_bazo(znamka["ime"], znamka["povezava"])
    for model1 in poisci_modele_req(znamka["povezava"]):
        razlicice = poisci_razlicice_req(model1["povezava"])
        model = razlicice[0]
        id_modela = shrani_model_v_bazo(
            model1["ime"],
            id_znamke,
            model["tip"],
            model["zacetno_leto"],
            model["motorji"],
            model1["povezava"]
        )
        for razlicica in razlicice[1]:
            id_razlicice = shrani_razlicico_v_bazo(
                razlicica["ime"],
                id_modela,
                razlicica["zacetno_leto"],
                razlicica["koncno_leto"],
                razlicica["motor"],
                razlicica["opis"],
                razlicica["povezava"],
            )